# Librerias

In [1]:
import pandas
import numpy
import os
import sys

In [2]:
import codigos_ene

# ENE

In [3]:
data_ene       = "C:/Users/et396/Dropbox/BASES/ENE-INEI/INEI/2015/Inicial"
output         = "C:/Users/et396/Dropbox/Docencia/Educate/Econometria/S2/Data"

In [4]:
# informacion de carpetas 
modulo_identifica    = "01_EMPRESA_IDENTIFICA.dta" # survey labor force
modulo_300           = "02_MODULO_I_II_III.dta"
modulo_400           = "04_MODULO_V_VI.dta"
modulo_900           = "05_MODULO_VII_VIII_IX.dta"

# Informacion de modulos
data_100      = os.path.join(data_ene, modulo_identifica)
data_300      = os.path.join(data_ene, modulo_300)
data_400      = os.path.join(data_ene, modulo_400)
data_900      = os.path.join(data_ene, modulo_900)

# Carga de data
#    data_empleo      = pandas.read_stata(data_100 , columns = variables_500 ) # filtro de variables
modulo_r100          = pandas.read_stata(data_100 ) # filtro de variables
modulo_r300          = pandas.read_stata(data_300 ) # filtro de variables
modulo_r400          = pandas.read_stata(data_400 ) # filtro de variables
modulo_r900          = pandas.read_stata(data_900 ) # filtro de variables

# Seleccion de variables
frame_r100 = codigos_ene.function_ene_r100(modulo_r100)
frame_r300 = codigos_ene.function_ene_r300(modulo_r300)
frame_r400 = codigos_ene.function_ene_r400(modulo_r400)
frame_r900 = codigos_ene.function_ene_r900(modulo_r900)

var_r100 = ['riruc', 'rC20', 'rexper', 'rneduca', 'rmujer', 'rencuesta','rDpto']
var_r300 = ['riruc', 'rcredito', 'rorga', 'rL']
var_r400 = ['riruc', 'rexporta']
var_r900 = ['riruc', 'rventas','rpt','rci']

base_r100 = frame_r100[var_r100]
base_r300 = frame_r300[var_r300]
base_r400 = frame_r400[var_r400]
base_r900 = frame_r900[var_r900]

# Union de bases de datos
base_final =pandas.merge(base_r100  , base_r300  , how='inner')
base_final =pandas.merge(base_final , base_r400  , how='inner')
base_final =pandas.merge(base_final , base_r900  , how='inner')

base_final = base_final[base_final.rencuesta==1] # solo encuesta completa
base_final = base_final[base_final['rL']>0]      # solo trabajadores mayores a cero
base_final = base_final[base_final['rventas']>0]      # solo ventas mayores a ceros 

base_final['rpl'] = base_final['rventas'] / base_final['rL'] # productividad
base_final['lnrpl'] = numpy.log(base_final['rpl'])

# Exprtar informacion
os.chdir(output) # exportar los resultados hacia una base de datos
base_final.to_csv("ENE_2015.csv") # Informacion de csv
base_final.to_stata("ENE_2015.dta") # Informacion de stata


C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S2\Data\codigos_ene.py:30: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['rneduca'] = frame['C12'].replace({"Sin nivel": "inicial",


In [5]:
base_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9457 entries, 1 to 19201
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   riruc      9457 non-null   object  
 1   rC20       9457 non-null   object  
 2   rexper     9457 non-null   float64 
 3   rneduca    9457 non-null   category
 4   rmujer     9457 non-null   int32   
 5   rencuesta  9457 non-null   int32   
 6   rDpto      9457 non-null   object  
 7   rcredito   9457 non-null   int32   
 8   rorga      9457 non-null   int32   
 9   rL         9457 non-null   float64 
 10  rexporta   9457 non-null   int32   
 11  rventas    9457 non-null   float64 
 12  rpt        9457 non-null   float64 
 13  rci        9457 non-null   float64 
 14  rpl        9457 non-null   float64 
 15  lnrpl      9457 non-null   float64 
dtypes: category(1), float64(7), int32(5), object(3)
memory usage: 1006.9+ KB


In [6]:
#modulo_r100.info()
#modulo_r100['C13B'].head(n=9)

In [7]:
base_final.head()

,riruc,rC20,rexper,rneduca,rmujer,rencuesta,rDpto,rcredito,rorga,rL,rexporta,rventas,rpt,rci,rpl,lnrpl
1,00000011374,Large,10.0,Univ.,0,1,Lima,0,0,59.0,0,1600800.0,1600800.0,790000.0,27132.203390,10.208477
4,00000051376,Micro,5.0,Univ.,0,1,Lima,0,0,4.0,0,140000.0,140000.0,45839.0,35000.000000,10.463103
16,00000051383,Large,6.0,Univ.,0,1,Lima,1,0,10.0,0,680000.0,680000.0,36910.0,68000.000000,11.127263
22,00000021702,Large,16.0,secundaria,0,1,Lima,0,0,12.0,0,717028.0,717028.0,385755.0,59752.333333,10.997964
25,00000051334,Micro,9.0,Univ.,0,1,Lima,0,0,8.0,0,164000.0,164000.0,17448.0,20500.000000,9.928180


In [8]:
base_final.describe().round(3).T

,count,mean,std,min,25%,50%,75%,max
rexper,9457.0,13.015,10.218,2.000,7.000,10.000,16.000,1.460000e+02
rmujer,9457.0,0.275,0.446,0.000,0.000,0.000,1.000,1.000000e+00
rencuesta,9457.0,1.000,0.000,1.000,1.000,1.000,1.000,1.000000e+00
rcredito,9457.0,0.391,0.488,0.000,0.000,0.000,1.000,1.000000e+00
rorga,9457.0,0.114,0.318,0.000,0.000,0.000,0.000,1.000000e+00
rL,9457.0,20.229,44.936,1.000,5.000,10.000,21.000,2.457000e+03
rexporta,9457.0,0.026,0.158,0.000,0.000,0.000,0.000,1.000000e+00
rventas,9457.0,725509.928,1758707.351,1000.000,132274.000,285799.000,650000.000,9.595844e+07
rpt,9457.0,638870.979,1522351.874,331.000,100000.000,240000.000,564061.000,6.450000e+07
rci,9457.0,285052.928,877003.772,0.000,31660.000,84900.000,232230.000,4.445092e+07
